# Log-Level-Classifier

## Description

The following notebook provides a small text classification model trained on OpenShift infrastructure logs. The model is tuned to classify log messages in terms of log level based on the log message sentiment.

## Step 1: Prepare how to parse log message and level from Loki queried logs

First things first, we need to write a couple of helper functions to extract logs originating from the following LogQL query against your OpenShift Logging instance:

```sh
export LOKI_PUBLIC_URL=$(k get route lokistack-dev -o jsonpath="{.spec.host}")

logcli --bearer-token="$(oc whoami -t)" \
       --addr "https://${LOKI_PUBLIC_URL}/api/logs/v1/infrastructure" \
       query '{kubernetes_namespace_name=~"openshift.*"} |~ ".*error.*"' \
       --output=jsonl \
       --limit=100000 | awk '{print > "file_" NR ".json"}'
```

In [20]:
import json
from pprint import pprint
from logfmt import parse
from fastai.text.all import *
import random

cur_dir = Path.cwd()
samples_dir = cur_dir/'samples'

def read_log_message(f):
    log_file = Path.read_json(f)
    log_line=json.loads(log_file['line'])
    log_data = {}
    for data in parse(io.StringIO(log_line['message'])):
        log_data.update(data)
    return log_data['msg'] if hasattr(log_data, 'msg') else log_line['message']

def read_log_level(f):
    log_file = Path.read_json(f)
    log_line=json.loads(log_file['line'])
    return log_line['level']
    
def get_logs(p):
    return get_files(p, extensions=['.json'])

## Step 2: Setup a DataBlock object and show batch results for verification

Setting up the `Datablock` instance is essential the first step to configure how we are going to read the log files:
1. `get_items` is the callback to read all json files from our `samples/train` directory
2. `get_x` is the callback to parse each log file (i.e. JSON) and extract the message from it. (Assumes `viaQ` format).
3. `get_y` is the callback to parse each log file (i.e. JSON) and extract the emitted level. This is info is used as a label for training.

__Note:__ For debugging one can use either the `DataBlock.summary` or `DataLoaders.show_batch` methods. Both provide valuable info into the pipeline is setup and how the batches look like.

In [21]:
logs = DataBlock(blocks=(TextBlock.from_folder(samples_dir), CategoryBlock),
                 get_items=get_logs,
                 get_y=read_log_level,
                 get_x=read_log_message,
                 splitter=RandomSplitter(valid_pct=0.1))

logs.summary(cur_dir, bs=4, show_batch=True)
dls=logs.dataloaders(samples_dir, verbose=True)
dls.show_batch(max_n=4)

Setting-up type transforms pipelines
Found 21683 items
2 datasets of sizes 19515,2168
Setting up Pipeline: read_log_message -> Tokenizer -> Numericalize
Setting up Pipeline: read_log_level -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: read_log_message -> Tokenizer -> Numericalize
    starting from
      /home/ptsiraki/workspace/jupyter/loglevelclassifier/samples/train/file_3031.json
    applying read_log_message gives
      time="2024-03-20T08:17:52Z" level=error msg="errored with condition: CredentialsProvisionFailure" controller=credreq cr=openshift-cloud-credential-operator/openshift-image-registry secret=openshift-image-registry/installer-cloud-credentials
    applying Tokenizer gives
      ['xxbos', 'time="2024', '-', '03', '-', '20t08:17:52z', '"', 'level', '=', 'error', 'msg="errored', 'with', 'condition', ':', 'credentialsprovisionfailure', '"', 'controller', '=', 'credreq', 'cr', '=', 'openshift', '-', 'cloud', '-', 'credentia

,text,category
0,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info
1,xxbos xxup xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,error
2,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,error
3,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,error


Found 21683 items
2 datasets of sizes 19515,2168
Setting up Pipeline: read_log_message -> Tokenizer -> Numericalize
Setting up Pipeline: read_log_level -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: Pad_Chunk -- {'pad_idx': 1, 'pad_first': True, 'seq_len': 72}
Setting up after_batch: Pipeline: 


,text,category
0,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info
1,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj,info
2,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxrep xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj,info
3,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,error


## Step 3: Execute the learning and finetuning stage

At this stage we use a pre-trained model (See [AWD_LSTM]()) and setup the `text_classifier_learner` to track `accuracy` as a learning performance metric. The model is fine tuned over 4 epochs.

__Note:__ The training over a dataset of ~20000 log files takes roundabout ~30min using CPU only as a PyTorch device.

In [9]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.651298,0.484627,0.767528,02:37


epoch,train_loss,valid_loss,accuracy,time
0,0.588837,0.336377,0.784594,08:01
1,0.529455,0.233636,0.988007,08:10
2,0.433399,0.157189,0.980166,07:47
3,0.525914,0.170646,0.988930,07:41


## Step 4 (Optional): Gather insighes on how well the learner did

In [10]:
intp = Interpretation.from_learner(learn)
intp.plot_top_losses(9)

,input,target,predicted,probability,loss
0,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.9046143889427185,4.239920139312744
1,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.9046143889427185,2.922067642211914
2,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.9015762209892273,2.3514485359191895
3,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.8995739221572876,2.3514485359191895
4,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.8995739221572876,2.336737871170044
5,xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,error,0.8995739221572876,2.3207948207855225
6,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,warn,0.7624432444572449,2.3012232780456543
7,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,info,warn,0.826315701007843,2.3012232780456543
8,xxbos xxup xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,error,warn,0.8656915426254272,2.3012232780456543


## Step 5: Use the learner to predict log level on new log lines

In [19]:
learn.predict('error in getting tenant secret data event="createOrUpdate" lokistack=map[Name:lokistack-dev Namespace:openshift-logging] _error=map[cause:map[ErrStatus:map[code:404 details:map[kind:Secret name:lokistack-dev-gateway] message:Secret "lokistack-dev-gateway" not found metadata:map[] reason:NotFound status:Failure]] msg:couldn\'t find tenant secret.]')

('error', tensor(1), tensor([1.2207e-04, 9.2125e-01, 7.8620e-02, 1.1597e-05]))